In [ ]:
import re, smtplib, requests
import dns.resolver

In [ ]:
def is_valid_email_format(email):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(regex, email) is not None

def get_ip_geolocation(ip_address):
    url = f"https://ipinfo.io/{ip_address}/json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    
    except requests.RequestException as e:
        print(f"Error retrieving geolocation information: {e}")
        return None
    
def get_dns_records(domain):
    records = {}

    dns_record_list = ['A', 'AAAA', 'MX', 'TXT', 'NS', 'SOA', 'CNAME', 'SPF', 'PTR', 'SRV', 'CERT', 'CAA']
    for row in dns_record_list:
        try:
            tmp_records = dns.resolver.resolve(domain, row)
            if row in ['MX']:
                records[row] = [tmp.exchange.to_text() for tmp in tmp_records]
            elif row in ['A', 'AAAA']:
                records[row] = [tmp.address for tmp in tmp_records]
            else:
                records[row] = [tmp.to_text() for tmp in tmp_records]
        except Exception as e:
            records[row] = f"Error: {e}"
    return records

def verify_email(email):
    if not is_valid_email_format(email):
        print("Invalid email format")
        return False, "Invalid email format"
    
    domain = email.split('@')[1]
    dns_records = get_dns_records(domain)
    if not dns_records:
        print("No DNS records found for domain")
        return False, "No DNS records found for domain"
    
    try:
        ipv4_location = get_ip_geolocation(dns_records['A'][0])
        dns_records['ipv4_location'] = ipv4_location
    except Exception as e:
        dns_records['ipv4_location'] = None

    try:
        ipv6_location = get_ip_geolocation(dns_records['AAAA'][0])
        dns_records['ipv6_location'] = ipv6_location
    except Exception as e:
        dns_records['ipv6_location'] = None
    
    # Try connecting to each MX server
    for mx in dns_records['MX']:
        try:
            server = smtplib.SMTP(mx)
            server.set_debuglevel(0)
            server.helo()
            server.mail('test@example.com')
            code, message = server.rcpt(email)
            server.quit()
            if code == 250:
                dns_records['EMAIL_EXISTS'] = True
                return True, dns_records
        except Exception as e:
            print(f"Error verifying email: {e}")
    dns_records['EMAIL_EXISTS'] = False
    return True, dns_records

In [ ]:
# Example usage
email = "benvanceuf@gmail.com"
dns_high_level, dns_details = verify_email(email)

In [ ]:
dns_high_level

In [ ]:
dns_details